# Routing optimization - Matoga
> Wymagania:
> * wirtualne środowisko: [environment dependencies](https://github.com/GIScience/openrouteservice-examples#local-installation)
> * klucz API: [openrouteservice API key](https://openrouteservice.org/dev/#/signup)

Routing optimization generally solves the [Vehicle Routing Problem](https://en.wikipedia.org/wiki/Vehicle_routing_problem)
(a simple example being the more widely known [Traveling Salesman Problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem)).
A more complex example would be the distribution of goods by a fleet of multiple vehicles to dozens of locations,
where each vehicle has certain time windows in which it can operate and each delivery location has certain time windows
in which it can be served (e.g. opening times of a supermarket).

Scenariusz: **dystrybucja przesyłek przez firmę kurierską na terenie Krakowa**

We'll solve this complex problem with the **optimization** endpoint of [openrouteservice](https://openrouteservice.org).

Import wymaganych bibliotek

In [11]:
import folium
from folium.plugins import BeautifyIcon
import pandas as pd
import openrouteservice as ors

## Logistyka

In total 20 sites were identified in need of the medical supplies, while 3 vehicles were scheduled for delivery.
Let's assume there was only one type of goods, e.g. standard moving boxes full of one medication.
(In reality there were dozens of different good types, which can be modelled with the same workflow,
but that'd unnecessarily bloat this example).

The **vehicles** were all located in the port of Beira and had the same following constraints:

- operation time windows from 8:00 to 20:00
- loading capacity of 300 *[arbitrary unit]*

The **delivery locations** were mostly located in the Beira region, but some extended ~ 200 km to the north of Beira.
Their needs range from 10 to 148 units of the arbitrary medication goods
(consult the file located at `../resources/data/idai_health_sites.csv`). Let's look at it in a map.

In [12]:
# parametry
ORS_API_KEY = '5b3ce3597851110001cf6248fc086db5e9964eaa9e562c4f6f927468'

N_VEHICLES = 4

In [13]:
# First define the map centered around Beira
m = folium.Map(location=[-18.63680, 34.79430], tiles='cartodbpositron', zoom_start=8)

# Next load the delivery locations from CSV file at ../resources/data/idai_health_sites.csv
# ID, Lat, Lon, Open_From, Open_To, Needed_Amount
deliveries_data = pd.read_csv(
    '../resources/data/idai_health_sites.csv',
    index_col="ID",
    parse_dates=["Open_From", "Open_To"]
)

# Plot the locations on the map with more info in the ToolTip
for location in deliveries_data.itertuples():
    tooltip = folium.map.Tooltip("<h4><b>ID {}</b></p><p>Supplies needed: <b>{}</b></p>".format(
        location.Index, location.Needed_Amount
    ))

    folium.Marker(
        location=[location.Lat, location.Lon],
        tooltip=tooltip,
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(location.Index),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(m)

# The vehicles are all located at the port of Beira
depot = [-19.818474, 34.835447]

folium.Marker(
    location=depot,
    icon=folium.Icon(color="green", icon="bus", prefix='fa'),
    setZIndexOffset=1000
).add_to(m)

m

## The routing problem setup

Now that we have described the setup sufficiently, we can start to set up our actual Vehicle Routing Problem.
For this example we're using the FOSS library of [Vroom](https://github.com/VROOM-Project/vroom), which has
[recently seen](http://k1z.blog.uni-heidelberg.de/2019/01/24/solve-routing-optimization-with-vroom-ors/) support for
openrouteservice and is available through our APIs.

To properly describe the problem in algorithmic terms, we have to provide the following information:

- **vehicles start/end address**: vehicle depot in Beira's port
- **vehicle capacity**: 300
- **vehicle operational times**: 08:00 - 20:00
- **service location**: delivery location
- **service time windows**: individual delivery location's time window
- **service amount**: individual delivery location's needs

We defined all these parameters either in code above or in the data sheet located in
`../resources/data/idai_health_sites.csv`.
Now we have to only wrap this information into our code and send a request to openrouteservice optimization service at
[`https://api.openrouteservice.org/optimization`](https://openrouteservice.org/dev/#/api-docs/optimization/post).

In [14]:
# Define the vehicles
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Vehicle
vehicles = list()
for idx in range(N_VEHICLES):
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx,
            start=list(reversed(depot)),
            # end=list(reversed(depot)),
            capacity=[300],
            time_window=[1553241600, 1553284800]  # Fri 8-20:00, expressed in POSIX timestamp
        )
    )

# Next define the delivery stations
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job
deliveries = list()
for delivery in deliveries_data.itertuples():
    deliveries.append(
        ors.optimization.Job(
            id=delivery.Index,
            location=[delivery.Lon, delivery.Lat],
            service=1200,  # Assume 20 minutes at each site
            amount=[delivery.Needed_Amount],
            time_windows=[[
                int(delivery.Open_From.timestamp()),  # VROOM expects UNIX timestamp
                int(delivery.Open_To.timestamp())
            ]]
        )
    )

With that set up we can now perform the actual request and let openrouteservice calculate the optimal vehicle schedule
for all deliveries.

In [15]:
# Initialize a client and make the request
ors_client = ors.Client(key=ORS_API_KEY)  # Get an API key from https://openrouteservice.org/dev/#/signup
result = ors_client.optimization(
    jobs=deliveries,
    vehicles=vehicles,
    geometry=True
)

# Add the output to the map
colors = ['green', 'red', 'blue', 'orange', 'black', 'yellow', 'pink', 'brown', 'gray', 'white']

for color, route in zip(colors[:N_VEHICLES], result['routes']):
    decoded = ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
    gj = folium.GeoJson(
        name='Vehicle {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                         "geometry": decoded,
                                                         "properties": {"color": color}
                                                         }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    gj.add_child(folium.Tooltip(
        """<h4>Vehicle {vehicle}</h4>
        <b>Distance</b> {distance} m <br>
        <b>Duration</b> {duration} secs
        """.format(**route)
    ))
    gj.add_to(m)

folium.LayerControl().add_to(m)
m

ApiError: 413 ({'code': 4, 'error': 'Too many vehicles ( 4 ) in query, maximum is set to 3'})

## Data view

Plotting it on a map is nice, but let's add a little more context to it in form of data tables.
First the overall trip schedule:

### Overall schedule

In [5]:
# Only extract relevant fields from the response
extract_fields = ['distance', 'amount', 'duration']
data = [{key: route[key] for key in extract_fields} for route in result['routes']]

vehicles_df = pd.DataFrame(data)
vehicles_df.index.name = 'vehicle'
vehicles_df

,distance,amount,duration
vehicle,,,
0,339333,[299],25798
1,474543,[295],23615
2,473790,[286],28710


So every vehicle's capacity is almost fully exploited. That's good.
How about a look at the individual service stations:

In [6]:
# Create a list to display the schedule for all vehicles
stations = list()
for route in result['routes']:
    vehicle = list()
    for step in route["steps"]:
        vehicle.append(
            [
                step.get("job", "Depot"),  # Station ID
                step["arrival"],  # Arrival time
                step["arrival"] + step.get("service", 0),  # Departure time

            ]
        )
    stations.append(vehicle)

Now we can look at each individual vehicle's timetable:

### Vehicle 0

In [7]:
df_stations_0 = pd.DataFrame(stations[0], columns=["Station ID", "Arrival", "Departure"])
df_stations_0['Arrival'] = pd.to_datetime(df_stations_0['Arrival'], unit='s')
df_stations_0['Departure'] = pd.to_datetime(df_stations_0['Departure'], unit='s')
df_stations_0

,Station ID,Arrival,Departure
0,Depot,2019-03-22 08:07:31,2019-03-22 08:07:31
1,5,2019-03-22 08:20:23,2019-03-22 08:40:23
2,4,2019-03-22 08:45:10,2019-03-22 09:05:10
3,6,2019-03-22 09:19:10,2019-03-22 09:39:10
4,14,2019-03-22 10:00:00,2019-03-22 10:20:00
5,10,2019-03-22 10:50:58,2019-03-22 11:10:58
6,8,2019-03-22 12:47:16,2019-03-22 13:07:16
7,17,2019-03-22 14:58:30,2019-03-22 15:18:30
8,1,2019-03-22 17:37:29,2019-03-22 17:57:29
9,Depot,2019-03-22 17:57:29,2019-03-22 17:57:29


### Vehicle 1

In [8]:
df_stations_1 = pd.DataFrame(stations[1], columns=["Station ID", "Arrival", "Departure"])
df_stations_1['Arrival'] = pd.to_datetime(df_stations_1['Arrival'], unit='s')
df_stations_1['Departure'] = pd.to_datetime(df_stations_1['Departure'], unit='s')
df_stations_1

,Station ID,Arrival,Departure
0,Depot,2019-03-22 09:44:18,2019-03-22 09:44:18
1,7,2019-03-22 10:00:00,2019-03-22 10:20:00
2,12,2019-03-22 10:58:47,2019-03-22 11:18:47
3,13,2019-03-22 11:32:04,2019-03-22 11:52:04
4,20,2019-03-22 17:17:53,2019-03-22 17:37:53
5,Depot,2019-03-22 17:37:53,2019-03-22 17:37:53


### Vehicle 2

In [9]:
df_stations_2 = pd.DataFrame(stations[2], columns=["Station ID", "Arrival", "Departure"])
df_stations_2['Arrival'] = pd.to_datetime(df_stations_2['Arrival'], unit='s')
df_stations_2['Departure'] = pd.to_datetime(df_stations_2['Departure'], unit='s')
df_stations_2

,Station ID,Arrival,Departure
0,Depot,2019-03-22 08:00:00,2019-03-22 08:00:00
1,3,2019-03-22 08:16:30,2019-03-22 08:36:30
2,2,2019-03-22 08:42:01,2019-03-22 09:02:01
3,18,2019-03-22 09:14:25,2019-03-22 09:34:25
4,11,2019-03-22 09:57:35,2019-03-22 10:17:35
5,9,2019-03-22 11:01:41,2019-03-22 11:21:41
6,16,2019-03-22 16:58:40,2019-03-22 17:18:40
7,15,2019-03-22 17:58:30,2019-03-22 18:18:30
8,Depot,2019-03-22 18:18:30,2019-03-22 18:18:30
